In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
import IPython

In [7]:
import csv
from scipy.signal import lfilter
from scipy.signal.windows import hamming
from scipy.interpolate import interp1d

# データをリサンプリングする関数
def resample_features(data, target_length):
    x_old = np.linspace(0, 1, data.shape[0])
    x_new = np.linspace(0, 1, target_length)
    interpolator = interp1d(x_old, data, axis=0, kind='linear')
    return interpolator(x_new)

# コラムの設定
data = []
fieldnames = []
fieldnames.append('Name')
fieldnames.append('Label')
fieldnames.append('Feature')
for i in range(0, 2584):
    fieldnames.append(str(i))
data.append(fieldnames)

# csvファイルにまとめる
# 取得するメル周波数ケプストラム係数の数
n_mfcc = 20
# 取得するLPCケプストラム係数の数
lpc_order = 20

label_dic = {'joyful': 0, 'fearful': 1, 'sorrowful': 2, 'relaxing': 3}
# label_dic = {'positive': 0, 'negative': 1}
label = 'relaxing'
musical_scale = {0:'C', 1:'C#', 2:'D', 3:'D#', 4:'E', 5:'F', 6:'F#', 7:'G', 8:'G#', 9:'A', 10:'A#', 11:'B'}
tonnetz_label = {0:'Fifth', 1:'Minor Third', 2:'Major Third', 3:'Minor Triad', 4:'Major Triad', 5:'Tonic'}

for i in range(1,51):
    audio_name = label + str(i)
    audiopath = 'music_data/' + label + '/' + audio_name + '.mp3'
    audio, sr = librosa.load(audiopath, offset=0.0, duration=60.0)
    harmonic, percussive = librosa.effects.hpss(audio, margin=3.0)
    # フレームの長さとステップサイズ（サンプリングレートに依存）
    frame_length = int(0.050 * sr)  # 25 ms
    frame_step = int(0.020 * sr)    # 10 ms

    # Spectral based novelty curve / Percussive ver.
    Percussive = np.log(np.abs(librosa.stft(percussive))+1e-9)
    percussive_spectral_novelty = None
    n_freq, n_tf = Percussive.shape
    percussive_spectral_novelty = np.zeros(n_tf-1)
    for f in range(0, n_freq):
        tmp = Percussive[f,1:] - Percussive[f,:-1]
        tmp[tmp<0.0] = 0.0
        percussive_spectral_novelty += tmp
    percussive_spectral_novelty /= np.max(percussive_spectral_novelty)
    percussive_spectral_novelty = np.append(percussive_spectral_novelty, 0)
    
    
    audio_data = np.append(audio_name, label_dic[label])
    audio_data = np.append(audio_data, 'Spectral')
    audio_data = np.append(audio_data, percussive_spectral_novelty)
    data.append(audio_data)
    
    # Harmonic MFCC
    mfccs = librosa.feature.mfcc(y=harmonic, sr=sr, n_mfcc=n_mfcc)
    for j in range(1, n_mfcc+1):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Harmonic MFCC'+str(j))
        audio_data = np.append(audio_data, mfccs[:][j-1])
        data.append(audio_data)
        
    # Percussive MFCC
    mfccs = librosa.feature.mfcc(y=percussive, sr=sr, n_mfcc=n_mfcc)
    for j in range(1, n_mfcc+1):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Percussive MFCC'+str(j))
        audio_data = np.append(audio_data, mfccs[:][j-1])
        data.append(audio_data)

    # Harmonic LPCケプストラム係数
    lpcccs = []

    # オーディオをフレームに分割
    frames = librosa.util.frame(harmonic, frame_length=frame_length, hop_length=frame_step)
    for frame in frames.T:
        frame = frame * hamming(len(frame))  # ハミング窓の適用
        lpc_coef = librosa.lpc(frame, order=lpc_order)  # LPC係数の計算
        lpc_kepstra = np.fft.ifft(np.log(np.abs(np.fft.fft(lpc_coef, n=512))))  # ケプストラムの計算
        lpcccs.append(lpc_kepstra[:lpc_order+1])  # 最初の20個の係数を保存

    # LPCケプストラム係数の実部を取得
    lpcccs = np.abs(np.array(lpcccs))

    # LPCケプストラム係数をリサンプリング
    lpcccs = resample_features(lpcccs, 2584)
    lpcccs = lpcccs.T
    for j in range(0, lpc_order+1):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Harmonic LPCCC'+str(j))
        audio_data = np.append(audio_data, lpcccs[:][j])
        data.append(audio_data)

    # Perccusive LPCケプストラム係数
    lpcccs = []

    # オーディオをフレームに分割
    frames = librosa.util.frame(percussive, frame_length=frame_length, hop_length=frame_step)
    for frame in frames.T:
        frame = frame * hamming(len(frame))  # ハミング窓の適用
        lpc_coef = librosa.lpc(frame, order=lpc_order)  # LPC係数の計算
        lpc_kepstra = np.fft.ifft(np.log(np.abs(np.fft.fft(lpc_coef, n=512))))  # ケプストラムの計算
        lpcccs.append(lpc_kepstra[:lpc_order+1])  # 最初の20個の係数を保存

    # LPCケプストラム係数の実部を取得
    lpcccs = np.abs(np.array(lpcccs))

    # LPCケプストラム係数をリサンプリング
    lpcccs = resample_features(lpcccs, 2584)
    lpcccs = lpcccs.T
    for j in range(0, lpc_order+1):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Percussive LPCCC'+str(j))
        audio_data = np.append(audio_data, lpcccs[:][j])
        data.append(audio_data)

    # クロマ特徴量(通常音源 ver.)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    for j in range(0, chroma.shape[0]):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Normal Chroma ' + musical_scale[j])
        audio_data = np.append(audio_data, chroma[:][j])
        data.append(audio_data)

    # クロマ特徴量(Harmonic ver.)
    chroma = librosa.feature.chroma_stft(y=harmonic, sr=sr)
    for j in range(0, chroma.shape[0]):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, 'Harmonic Chroma ' + musical_scale[j])
        audio_data = np.append(audio_data, chroma[:][j])
        data.append(audio_data)

    # トーンネット特徴量の抽出
    tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
    for j in range(0, tonnetz.shape[0]):
        audio_data = np.append(audio_name, label_dic[label])
        audio_data = np.append(audio_data, tonnetz_label[j])
        audio_data = np.append(audio_data, tonnetz[:][j])
        data.append(audio_data)

with open(label + '_data_2.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for row in data:
        writer.writerow(row)

In [8]:
import pandas as pd
joyful_df = pd.read_csv('feature_data/joyful_data_2.csv')
joyful_df = joyful_df.fillna(0)
joyful_df.shape

(5650, 2587)